## Low-Level Arrays


An array is a group of related variables stored one after another in a contiguous portion of computer memory. We can use a string as an example, which Python stores as a sequence of individual characters.

The benefit of using this type of data structure is the ability to retrieve any information in constant time (O(1)). By knowing the memory location at which the array starts and the position of the desired information, you can simply calculate the memory address using the formula `start + (index * element size)` (Of course, the arithmetic for calculating memory addresses within an array can
be handled automatically).

![String Example](../img/arrays/string_example.png)
*Example of how a string is stored in memory as an array of characters.*

### Referential Arrays

referential arrays store the bits that represents the reference address memory of the data. 


In scenarios where we don't know the exact memory requirements for the variables we want to store in an array, allocating maximum memory for each variable would be inefficient and wasteful. Instead, we can utilize referential arrays, where we store references to the memory addresses where these variables are stored.

![Referential Array Example](../img/arrays/string_reference_array.png)
*Example illustrating a referential array storing references to strings.*

### Compact Arrays

compact arrays store the bits that represents the primary data. 

They have several advantages over referential structures in terms of computing performance, most significantly, the overall memory usage will be much lower for a compact structure because there is no overhead devoted to the explicit storage of the sequence of memory references.

![Compact Array Example](../img/arrays/string_compact_array.png)

*Example illustrating a compact array storing references to strings.*

#  Dynamic Arrays

When creating a low-level array in a computer system, the precise size of that array must be explicitly declared in order for the system to properly allocate a consecutive piece of memory for its storage.

The first key to providing the semantics of a dynamic array is that a dynamic array instance maintains an underlying array that often has greater capacity than the current length of the dynamic array. For example, if a user create an dynamic array with 10 elements, we can create an underlying array with 15 elements size (5 extra element sizes), than facilitating the user to append new itens to their dynamic array. 

If a user continues to append elements to a list, any reserved capacity will eventually be exhausted. In that case, the class requests a new, larger array from the system, and initializes the new array so that its prefix matches that of the existing smaller array.

### Code Fragment 5.1

An experiment to investigate the correlation between the length of a list and its underlying size in Python.

In this experiment, we observe that even an empty list consumes a certain amount of memory. Additionally, we note that the length of the list does not increase linearly as elements are added. Instead, the length only increases when attempting to insert more elements than the current capacity of the underlying array. At this point, a new underlying array is created with a larger capacity, and the elements are transferred to it. This process continues as more elements are added to the list.

In [1]:
#Code Fragment 5.1
import sys


data = []
for k in range(10):
    a = len(data)
    b = sys.getsizeof(data)
    print(f"Length: {a}; Size in bytes: {b}")
    data.append(None)
    

Length: 0; Size in bytes: 56
Length: 1; Size in bytes: 88
Length: 2; Size in bytes: 88
Length: 3; Size in bytes: 88
Length: 4; Size in bytes: 88
Length: 5; Size in bytes: 120
Length: 6; Size in bytes: 120
Length: 7; Size in bytes: 120
Length: 8; Size in bytes: 120
Length: 9; Size in bytes: 184


## Creating a Dynamic Array

The issue is to consider is how large of a new array to create. A commonly used rule is for the new array to have twice the capacity of the existing array that has been filled.

obs: Python ctypes is a foreign function library for Python that provides C compatible data types, and it allows calling functions in DLLs or shared libraries. It is a part of the Python standard library and provides a way to call functions in shared libraries or shared objects (.dll, .so).

In [18]:
#Code Fragment 5.3
import sys
from ctypes import sizeof, py_object

class DynamicArray:
    def __init__(self):
        self._n = 0
        self._capacity = 1
        self._A = self._make_array(self._capacity)

    def __len__(self):
        return self._n

    def __getitem__(self, position):
        if not 0 <= position < self._n:
            raise IndexError('Invalid Index')
        return self._A[position]

    def __sizeof__(self):
        return sizeof(self._A) + sys.getsizeof(self._n) + sys.getsizeof(self._capacity)

    def __repr__(self):
        return '[' + ','.join([str(self._A[i]) for i in range(self._n)]) + ']'

    def _resize(self, new_capacity):
        B = self._make_array(new_capacity)
        B[:self._n] = self._A[:self._n]
        self._A = B
        self._capacity = new_capacity
        
    def _make_array(self, capacity):
        return (capacity*py_object)()

    def append(self, obj):
        if self._n == self._capacity:
            self._resize(2 * self._capacity)
        self._A[self._n] = obj
        self._n += 1

In [56]:
my_array = DynamicArray()
for i in range(10):
    a = len(my_array)
    b = my_array.__sizeof__()
    print(f"Length: {a}; Size in bytes: {b}")
    my_array.append(None)

Length: 0; Size in bytes: 64
Length: 1; Size in bytes: 64
Length: 2; Size in bytes: 72
Length: 3; Size in bytes: 88
Length: 4; Size in bytes: 88
Length: 5; Size in bytes: 120
Length: 6; Size in bytes: 120
Length: 7; Size in bytes: 120
Length: 8; Size in bytes: 120
Length: 9; Size in bytes: 184


## Amortized Analysis of Dynamic Arrays

Amortized analysis is a method used to analyze the time or space complexity of algorithms or data structures over a sequence of operations. It provides a way to average out the cost of individual operations to understand the overall performance.

Consider the `append` operation on a dynamic array. When the array reaches its full capacity, it needs to resize by allocating a new array with double the size and copying existing elements. Although this resizing operation has a time complexity of O(n), it occurs infrequently relative to the number of append operations. As a result, the amortized time complexity of the `append` operation remains constant (O(1)).

Lets look deeper about the time complexity of each insertion on a Dynamic Array:

- **Appending with Available Capacity**: If there is available space in the underlying array, the `append` operation can be performed in constant time (O(1)). This involves simple pointer manipulation to add the element to the end of the array.

- **Resizing the Array**: When the underlying array is full, it needs to be resized. This involves allocating a new array with double the size and copying existing elements. This resizing operation is O(n), where n is the number of elements in the array.

So, to calculate the amount of time to compute N insertions we get on this case 

total_cost = $1 + 2 + 1 + 4 + 1 + 1 + 8 + 1 + 1 + 1 + 1 ... $

total_cost = $(2 + 4 + 8 ...) + N - \lfloor\log_2(N)\rfloor $

total_cost = $\sum_{k=1}^\left(\lfloor\log_2(N)\rfloor + 1\right) 2^k + N  - \lfloor\log_2(N)\rfloor $

total_cost = $\frac{2^\left(\lfloor\log_2(N)\rfloor + 1\right) - 1}{2 - 1} + N - \lfloor\log_2(N)\rfloor$

total_cost = $\frac{2N - 1}{1} + N - \lfloor\log_2(N)\rfloor$

total_cost = $3N - \lfloor\log_2(N)\rfloor - 1$

total_cost = O(N)

from here we can deduce a general formula where the previous rate 2 becomes the r factor of amortization

total_cost = $\sum_{k=1}^\left(\lfloor\log_r(N)\rfloor + 1\right) r^k + N  - \lfloor\log_r(N)\rfloor $

for example lets assume r =  1.5 (we increase 50% of the array length at each resize operation)

total_cost = $\sum_{k=1}^\left(\lfloor\log_\left(1.5\right)(N)\rfloor + 1 \right) (1.5)^k + N  - \lfloor\log_\left(1.5\right)(N)\rfloor $

total_cost = $\frac{(1.5)^\left(\lfloor\log_\left(1.5\right)(N)\rfloor + 1\right) - 1}{1.5 - 1} + N - \lfloor\log_\left(1.5\right)(N)\rfloor$

total_cost = $\frac{1.5N - 1}{0.5} + N - \lfloor\log_\left(1.5\right)(N)\rfloor$

total_cost = $4N - \lfloor\log_\left(1.5\right)(N)\rfloor - 2$

In [75]:
#Code Fragment 5.4
from time import time


def compute_average(n):
    data = []
    start = time()
    for k in range(n):
        data.append(None)
    end = time()
    return (end - start)/n

for i in range(1000, 1000000, 100000):
    print(f"valor de n: {i}, valor amortizado por operacao: {compute_average(i)/(10**-6)}")

valor de n: 1000, valor amortizado por operacao: 0.05269050598144531
valor de n: 101000, valor amortizado por operacao: 0.037679577818011296
valor de n: 201000, valor amortizado por operacao: 0.0344674978683244
valor de n: 301000, valor amortizado por operacao: 0.02854923869288245
valor de n: 401000, valor amortizado por operacao: 0.013344900269163518
valor de n: 501000, valor amortizado por operacao: 0.011377467842635044
valor de n: 601000, valor amortizado por operacao: 0.010732406387709937
valor de n: 701000, valor amortizado por operacao: 0.010700606755625336
valor de n: 801000, valor amortizado por operacao: 0.011496478401022161
valor de n: 901000, valor amortizado por operacao: 0.014143699281885145


# Efficiency of Python’s Sequence Types

### Adding Elements to a List
The insertion process in a dynamic array typically involves the following steps:

1. **Check Capacity**: Before inserting a new element, the dynamic array checks if there is enough capacity to accommodate the new element. If the array is full, it needs to be resized to create additional space.

2. **Shift Elements**: If inserting in the middle of the array, elements to the right of the insertion point need to be shifted to make space for the new element.

3. **Insert Element**: Once space is made, the new element is inserted into the desired position in the array.

4. **Resize Operation (if necessary)**: Similar to appending, if the array is full after insertion, it may need to be resized to accommodate more elements.

#### Time Complexity

The time complexity of inserting into the middle of a dynamic array involves both shifting elements and potential resizing:

- **Shifting Elements**: Shifting elements to make space for the new element in the middle of the array is O(K + 1).
- **Appending with Available Capacity**: If there's available space in the array after shifting, insertion can be performed in constant time (O(1)).
- **Resizing the Array**: If the array needs to be resized after insertion, the operation becomes O(n), similar to appending.


In [40]:
class DynamicArrayInsert(DynamicArray):
    def insert(self, position, obj):
        if self._n == self._capacity:
            self._resize(2*self._capacity)
        for j in range(self._n, position, -1):
            self._A[j] = self._A[j-1]
        self._A[position] = obj
        self._n += 1



In [177]:
from time import time


def compute_average(n, position):
    data = DynamicArrayInsert()
    start = time()
    count = 1
    for k in range(n):
        if position == 'final':
            position_value = count - 1
        elif position == 'middle':
            position_value = count // 2
        else:
            position_value = 0
        data.insert(position_value, None)
        count += 1
    end = time()
    return (end - start)/n

print("inserindo dados no final do array")
for i in range(10000, 50000, 10000):
    print(f"valor de n: {i}, valor amortizado por operacao: {compute_average(i, 'final')/(10**-6)} u segundos")
print("inserindo dados no meio do array")
for i in range(10000, 50000, 10000):
    print(f"valor de n: {i}, valor amortizado por operacao: {compute_average(i, 'middle')/(10**-6)} u segundos")
print("inserindo dados no inicio do array")
for i in range(10000, 50000, 10000):
    print(f"valor de n: {i}, valor amortizado por operacao: {compute_average(i, 'beguining')/(10**-6)} u segundos")

inserindo dados no final do array
valor de n: 10000, valor amortizado por operacao: 0.18770694732666016 u segundos
valor de n: 20000, valor amortizado por operacao: 0.17174482345581057 u segundos
valor de n: 30000, valor amortizado por operacao: 0.13644695281982422 u segundos
valor de n: 40000, valor amortizado por operacao: 0.14689564704895022 u segundos
inserindo dados no meio do array
valor de n: 10000, valor amortizado por operacao: 96.53053283691406 u segundos
valor de n: 20000, valor amortizado por operacao: 199.19607639312744 u segundos
valor de n: 30000, valor amortizado por operacao: 292.03499158223474 u segundos
valor de n: 40000, valor amortizado por operacao: 389.5691990852356 u segundos
inserindo dados no inicio do array
valor de n: 10000, valor amortizado por operacao: 197.5805997848511 u segundos
valor de n: 20000, valor amortizado por operacao: 384.4240427017212 u segundos
valor de n: 30000, valor amortizado por operacao: 581.8004687627157 u segundos
valor de n: 40000, 

The removal process in a dynamic array typically involves the following steps:

1. **Search for Element**: The dynamic array searches for the specified element to be removed. This can be done sequentially from the beginning of the array to the end, or using a more efficient search algorithm depending on the requirements.

2. **Identify Element Location**: Once the element to be removed is found, its location (index) in the array is identified.

3. **Shift Elements**: After identifying the location of the element to be removed, all subsequent elements to the right of the removed element need to be shifted to the left to fill the gap created by the removal.

4. **Update Size**: After shifting elements, the size of the array is decremented to reflect the removal of the element.


In [121]:
class DynamicArrayRemove(DynamicArray):
    def remove(self, obj):
        occurencies = 0
        for k in range(self._n):
            if self._A[k] == obj:
                occurencies += 1
                for j in range(k, self._n, -1):
                    self._A[j] = self._A[j+1]
                    self._A[self._n - occurencies] = None
        if not occurencies:
            raise ValueError('value not found')
        else:
            self._n -= occurencies


my_array = DynamicArrayRemove()
my_array.append(10)
my_array.append(10)
my_array.append(10)
print(my_array)
my_array.remove(10)
print(my_array)

[10,10,10]
[]


# Questions

## Question 1
```Execute the experiment from Code Fragment 5.1 and compare the results on your system to those we report in Code Fragment 5.2.```

In [1]:
import sys


data = []
for k in range(20):
    a = len(data)
    b = sys.getsizeof(data)
    print('Length: {0:3d}; Size in bytes: {1:4d}'.format(a,b))
    data.append(None)

Length:   0; Size in bytes:   56
Length:   1; Size in bytes:   88
Length:   2; Size in bytes:   88
Length:   3; Size in bytes:   88
Length:   4; Size in bytes:   88
Length:   5; Size in bytes:  120
Length:   6; Size in bytes:  120
Length:   7; Size in bytes:  120
Length:   8; Size in bytes:  120
Length:   9; Size in bytes:  184
Length:  10; Size in bytes:  184
Length:  11; Size in bytes:  184
Length:  12; Size in bytes:  184
Length:  13; Size in bytes:  184
Length:  14; Size in bytes:  184
Length:  15; Size in bytes:  184
Length:  16; Size in bytes:  184
Length:  17; Size in bytes:  248
Length:  18; Size in bytes:  248
Length:  19; Size in bytes:  248


## Question 2
```In Code Fragment 5.1, we perform an experiment to compare the length of a Python list to its underlying memory usage. Determining the sequence of array sizes requires a manual inspection of the output of that program. Redesign the experiment so that the program outputs only those values of k at which the existing capacity is exhausted. For example, on a system consistent with the results of Code Fragment 5.2, your program should output that the sequence of array capacities are 0, 4, 8, 16, 24, . . . .```

In [4]:
import sys


data = []
for k in range(50):
    a = len(data)
    b = sys.getsizeof(data)
    total = (b-56)/8
    if total == a:
        print(a)
    data.append(None)

0
4
8
16
24
32
40


## Question 3
```Modify the experiment from Code Fragment 5.1 in order to demonstrate that Python’s list class occasionally shrinks the size of its underlying array when elements are popped from a list.```

In [6]:
import sys


data = []
for k in range(20):
    data.append(None)

for k in range(20):
    data.pop()
    a = len(data)
    b = sys.getsizeof(data)
    print('Length: {0:3d}; Size in bytes: {1:4d}'.format(a,b))

Length:  19; Size in bytes:  248
Length:  18; Size in bytes:  248
Length:  17; Size in bytes:  248
Length:  16; Size in bytes:  248
Length:  15; Size in bytes:  248
Length:  14; Size in bytes:  248
Length:  13; Size in bytes:  248
Length:  12; Size in bytes:  248
Length:  11; Size in bytes:  184
Length:  10; Size in bytes:  184
Length:   9; Size in bytes:  184
Length:   8; Size in bytes:  184
Length:   7; Size in bytes:  152
Length:   6; Size in bytes:  152
Length:   5; Size in bytes:  120
Length:   4; Size in bytes:  120
Length:   3; Size in bytes:  120
Length:   2; Size in bytes:  120
Length:   1; Size in bytes:   88
Length:   0; Size in bytes:   56


## Question 4
```Our DynamicArray class, as given in Code Fragment 5.3, does not support use of negative indices with getitem . Update that method to better match the semantics of a Python list.```

In [35]:
class DynamicArrayNegativeIndex(DynamicArray):
    def __getitem__(self, position):
        if position < 0:
            position = self._n + position
        if not 0 <= position < self._n:
            raise IndexError('Invalid Index')
        return self._A[position]

In [36]:
my_array = DynamicArrayNegativeIndex()

for i in range(10):
    my_array.append(i)

print(my_array[-1])

9


## Question 5
```Redo the justification of Proposition 5.1 assuming that the the cost of growing the array from size k to size 2k is 3k cyber-dollars. How much should each append operation be charged to make the amortization work?```

total_cost = $1 + 8 + 1 + 16 + 1 + 1 + 1 + 24 + 1 + 1 + 1 + 1 ... $

total_cost = $3(2 + 4 + 8 ...) + N - \lfloor\log_2(N)\rfloor $

total_cost = $3\sum_{k=1}^\left(\lfloor\log_2(N)\rfloor + 1 \right) 2^k + N  - \lfloor\log_2(N)\rfloor $

total_cost = $3\frac{2^\left(\lfloor\log_2(N)\rfloor + 1 \right) - 1}{2 - 1} + N - \lfloor\log_2(N)\rfloor$

total_cost = $3\frac{2N - 1}{1} + N - \lfloor\log_2(N)\rfloor$

total_cost = $7N - \lfloor\log_2(N)\rfloor - 3$

## Question 6
```Our implementation of insert for the DynamicArray class, as given in Code Fragment 5.5, has the following inefficiency. In the case when a re-size occurs, the resize operation takes time to copy all the elements from an old array to a new array, and then the subsequent loop in the body of  insert shifts many of those elements. Give an improved implementation of the insert method, so that, in the case of a resize, the elements are shifted into their final position during that operation, thereby avoiding the subsequent shifting```

In [175]:
class OptimizedDynamicInsert(DynamicArrayInsert):
    def _shift(self, k, target_array=None):
        array_to_shift = target_array if target_array else self._A
        for i in range(self._n, k, -1):
            array_to_shift[i] = self._A[i-1]
            
    def _resize(self, c, k):
        B = self._make_array(c)
        for i in range(k):
            B[i] = self._A[i]
        self._shift(k, B)
        self._A = B
        self._capacity = c
        
    def insert(self, value, k):
        if self._n == self._capacity:
            self._resize(2*self._capacity, k)
        else:
            self._shift(k)
        self._A[k] = value
        self._n += 1
    
    def append(self, obj):
        if self._n == self._capacity:
            self._resize(2 * self._capacity, self._n)
        self._A[self._n] = obj
        self._n += 1

In [178]:
array = OptimizedDynamicInsert()

array.insert(10,0)
array.insert(10,0)
array.insert(10,0)
array.insert(100,2)
print(array)

[10,10,100,10]


## Question 7
```Let A be an array of size n ≥ 2 containing integers from 1 to n − 1, inclusive, with exactly one repeated. Describe a fast algorithm for finding the integer in A that is repeated.```

In [186]:
#O(n) algorithm

def found_duplicate(array):
    n = len(a)
    b = [i for i in range(1,n)]
    for i in a:
        try:
            b.remove(i)
        except Exception as e:
            return i
    return

a  =  [1,4,2,3,4,5]
found_duplicate(a)

4

## Question 8

```Experimentally evaluate the efficiency of the pop method of Python’s list class when using varying indices as a parameter, as we did for insert. Report your results akin to Table 5.5```

In [192]:
class DynamicArrayPop(DynamicArray):
    def pop(self, k=None):
        if not k:
            k = self._n - 1
        for i in range(k, self._n - 1):
            self._A[i] = self._A[i + 1]
        self._n -= 1
        self._A[self._n] = None
        if self._n < self._capacity//4:
            self._resize(self._capacity//2)

In [198]:
from time import time


def compute_average(n, position):
    data = [0]*n
    start = time()
    for k in range(n):
        if position == 'final':
            position_value = len(data) - 1
        elif position == 'middle':
            position_value = len(data) // 2
        else:
            position_value = 0
        data.pop(position_value)
    end = time()
    return (end - start)/n

print("removendo dados no final do array")
for i in range(100000, 500000, 100000):
    print(f"valor de n: {i}, valor amortizado por operacao: {compute_average(i, 'final')/(10**-6)} u segundos")
print("removendo dados no meio do array")
for i in range(100000, 500000, 100000):
    print(f"valor de n: {i}, valor amortizado por operacao: {compute_average(i, 'middle')/(10**-6)} u segundos")
print("removendo dados no inicio do array")
for i in range(100000, 500000, 100000):
    print(f"valor de n: {i}, valor amortizado por operacao: {compute_average(i, 'beguining')/(10**-6)} u segundos")

removendo dados no final do array
valor de n: 100000, valor amortizado por operacao: 0.038518905639648444 u segundos
valor de n: 200000, valor amortizado por operacao: 0.0443720817565918 u segundos
valor de n: 300000, valor amortizado por operacao: 0.03778775533040365 u segundos
valor de n: 400000, valor amortizado por operacao: 0.04300832748413086 u segundos
removendo dados no meio do array
valor de n: 100000, valor amortizado por operacao: 2.848808765411377 u segundos
valor de n: 200000, valor amortizado por operacao: 5.613065958023072 u segundos
valor de n: 300000, valor amortizado por operacao: 9.7862180074056 u segundos
valor de n: 400000, valor amortizado por operacao: 15.593248605728151 u segundos
removendo dados no inicio do array
valor de n: 100000, valor amortizado por operacao: 5.423221588134766 u segundos
valor de n: 200000, valor amortizado por operacao: 14.392311573028564 u segundos
valor de n: 300000, valor amortizado por operacao: 28.809954325358074 u segundos
valor de 